In [10]:
#!pip install kaggle
#!mkdir ~/.kaggle
#!chmod 600 /root/.kaggle/kaggle.json

In [12]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [13]:
api = KaggleApi()
api.authenticate()

In [ ]:
#enroll to the competition from the site
api.competition_download_file('sentiment-analysis-on-movie-reviews', 'test.tsv.zip', path='./')
api.competition_download_file('sentiment-analysis-on-movie-reviews', 'train.tsv.zip', path='./')



In [21]:
import zipfile
with zipfile.ZipFile("./test.tsv.zip", 'r') as zipref:
  zipref.extractall("./")

import zipfile
with zipfile.ZipFile("./train.tsv.zip", 'r') as zipref:
  zipref.extractall("./")

In [22]:
###### pre process the data! ################
import pandas as pd

In [ ]:
df = pd.read_csv( "train.tsv", sep='\t');
df.head()

In [ ]:
#df.drop_duplicates(subset=['SentenceId'], keep='first' )

In [ ]:
df['Sentiment'].value_counts().plot(kind='bar')

In [28]:
seq_len = 512
num_samples = len(df)

In [ ]:
# !pip install transformers

In [3]:
from transformers import BertTokenizer
import numpy as np

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
tokens = tokenizer( df['Phrase'].tolist(), max_length=seq_len, truncation=True, padding='max_length', add_special_tokens=True, return_tensors='np')

In [ ]:
tokens.keys()

In [ ]:
tokens['input_ids'], tokens['attention_mask']

In [39]:
with open('movie-xids.npy', 'wb' ) as f:
  np.save(f, tokens['input_ids'])

with open('movie-xmask.npy', 'wb' ) as f:
  np.save(f, tokens['attention_mask'])

In [ ]:
arr = df["Sentiment"].values
labels = np.zeros( (num_samples, arr.max()+1) )
labels.shape

In [ ]:
labels[ np.arange(num_samples), arr] = 1
labels

In [47]:
with open('movie-labels.npy', 'wb') as f:
  np.save(f, labels)

In [46]:
del tokens

In [2]:
######### design pipline ####
import numpy as np

In [3]:
with open('movie-labels.npy', 'rb' ) as f:
  labels = np.load(f, allow_pickle=True);

with open('movie-xmask.npy', 'rb' ) as f:
  xmask = np.load(f, allow_pickle=True);

with open('movie-xids.npy', 'rb' ) as f:
  xid = np.load(f, allow_pickle=True);

In [4]:
import tensorflow as tf

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((xid, xmask, labels))
dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.int64, name=None), TensorSpec(shape=(512,), dtype=tf.int64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [6]:
def map_func( input_ids, mask, lables ):
  return { 'input_ids':input_ids, 'attention_mask':mask}, labels

dataset = dataset.map(map_func)
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int64, name=None)}, TensorSpec(shape=(156060, 5), dtype=tf.float64, name=None))>

In [7]:
batch_size = 16
dataset = dataset.shuffle(10000).batch( batch_size, drop_remainder=True)
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 156060, 5), dtype=tf.float64, name=None))>

In [8]:
split =0.9
size = int( (xid.shape[0]/ batch_size)* split)
size

8778

In [9]:
train_dataset = dataset.take(size)
val_dataset = dataset.skip(size)

In [10]:
del dataset

In [ ]:
# tf.data.experimental.save( train_dataset, 'train')
# tf.data.experimental.save( val_dataset, 'val')

Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)},
 TensorSpec(shape=(16, 156060, 5), dtype=tf.float64, name=None))

In [12]:
val_dataset.element_spec == train_dataset.element_spec

True